Connected to venv-plots (Python 3.8.10)

In [102]:
import pandas as pd, numpy as np
import sys, os, os.path, tempfile, time, logging, json
import matplotlib.pyplot as plt, seaborn as sns # plotting

# univariate statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

# Dataset
from sklearn.datasets import make_classification

from itertools import product

# Joblib
from joblib import Parallel, delayed
from joblib import Memory
from joblib import cpu_count

# Metrics
import sklearn.metrics as metrics

# Resampling
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

# Set pandas display options
pd.set_option('display.max_colwidth', None)  # No maximum width for columns
pd.set_option('display.width', 1000)  # Set the total width to a large number

# inputs
ROOT ="/neurospin/signatures/2025_spetiton_rlink_predict_response_anat/"

################################################################################
# Read config file (Parameters)

In [103]:
config_file = ROOT+'models/study-rlink_mod-cat12vbm_type-roi+age+sex+site_lab-M00_v-4/supervised_classification_config.json'

with open(config_file) as json_file:
    config = json.load(json_file)

sys.path.append("/neurospin/psy_sbox/temp_sara/venv-plots/lib/python3.8/site-packages/")

# Utils
import nitk
from nitk.sys_utils import import_module_from_path
from nitk.pandas_utils.dataframe_utils import expand_key_value_column
from nitk.pandas_utils.dataframe_utils import describe_categorical
from nitk.ml_utils.dataloader_table import get_y, get_X
from nitk.ml_utils.residualization import get_residualizer

# Import models
sys.path.append(os.path.dirname(config["models_path"]))
root, _ = os.path.splitext(os.path.basename(config["models_path"]))

config['log_filename'] = os.path.splitext(config_file)[0] + ".log"
config['output_filename'] = os.path.splitext(config_file)[0] + "_scores.csv"#_no_balanced_weights.csv"
config['stratification_sse'] = os.path.splitext(config_file)[0] + "_stratification-sse.csv"
config['cv5test'] = os.path.splitext(config_file)[0] + "_cv5test.json"

config['cachedir'] = os.path.splitext(config_file)[0] + "/cachedir"

# Import the module
import_module_from_path(config["models_path"])
from classification_models import make_models

def print_log(*args):
    with open(config['log_filename'], "a") as f:
        print(*args, file=f)

print_log('###########################################################')
print_log(config)

################################################################################
# Read Data

In [71]:
data = pd.read_csv(config['input_filename'])
print(data)

################################################################################
# Target variable => y

    participant_id  3rd Ventricle_GM_Vol  4th Ventricle_GM_Vol  Right Accumbens Area_GM_Vol  Left Accumbens Area_GM_Vol  Right Amygdala_GM_Vol  Left Amygdala_GM_Vol  Brain Stem_GM_Vol  Right Caudate_GM_Vol  Left Caudate_GM_Vol  ...  Right TMP temporal pole_CSF_Vol  Left TMP temporal pole_CSF_Vol  Right TrIFG triangular part of the inferior frontal gyrus_CSF_Vol  Left TrIFG triangular part of the inferior frontal gyrus_CSF_Vol  Right TTG transverse temporal gyrus_CSF_Vol  Left TTG transverse temporal gyrus_CSF_Vol  age  sex     site    y
0        sub-50875              0.222878              0.231475                     0.430792                    0.473015               0.982001              0.963068           1.032879              3.101525             3.149932  ...                         3.480951                        3.405569                                                           1.074121                                                          1.165507                            

In [72]:
y = get_y(data, target_column=config['target'],
          remap_dict=config['target_remap'], print_log=print_log)

################################################################################
# X: Input Data
# Select Input = dataframe - (target + drop + residualization)

In [73]:
y = get_y(data, target_column=config['target'],
          remap_dict=config['target_remap'], print_log=print_log)
print(np.shape(y))
################################################################################
# X: Input Data
# Select Input = dataframe - (target + drop + residualization)

(117,)


In [74]:
data = pd.read_csv(config['input_filename'])
print(data)

################################################################################
# Target variable => y

    participant_id  3rd Ventricle_GM_Vol  4th Ventricle_GM_Vol  Right Accumbens Area_GM_Vol  Left Accumbens Area_GM_Vol  Right Amygdala_GM_Vol  Left Amygdala_GM_Vol  Brain Stem_GM_Vol  Right Caudate_GM_Vol  Left Caudate_GM_Vol  ...  Right TMP temporal pole_CSF_Vol  Left TMP temporal pole_CSF_Vol  Right TrIFG triangular part of the inferior frontal gyrus_CSF_Vol  Left TrIFG triangular part of the inferior frontal gyrus_CSF_Vol  Right TTG transverse temporal gyrus_CSF_Vol  Left TTG transverse temporal gyrus_CSF_Vol  age  sex     site    y
0        sub-50875              0.222878              0.231475                     0.430792                    0.473015               0.982001              0.963068           1.032879              3.101525             3.149932  ...                         3.480951                        3.405569                                                           1.074121                                                          1.165507                            

In [75]:
y = get_y(data, target_column=config['target'],
          remap_dict=config['target_remap'], print_log=print_log)
print(np.shape(y))
print(y[:10])
################################################################################
# X: Input Data
# Select Input = dataframe - (target + drop + residualization)

(117,)
0    0
1    1
2    0
3    0
4    0
5    0
6    0
7    1
8    1
9    0
Name: y, dtype: int64


In [76]:
input_columns = [c for c in data.columns if c not in [config['target']] + \
    config['drop'] + config['residualization']]

X = get_X(data, input_columns, print_log=print_log)
X = X.values

In [77]:
print(input_columns)
################################################################################
# Z: Residualization data

['3rd Ventricle_GM_Vol', '4th Ventricle_GM_Vol', 'Right Accumbens Area_GM_Vol', 'Left Accumbens Area_GM_Vol', 'Right Amygdala_GM_Vol', 'Left Amygdala_GM_Vol', 'Brain Stem_GM_Vol', 'Right Caudate_GM_Vol', 'Left Caudate_GM_Vol', 'Right Cerebellum Exterior_GM_Vol', 'Left Cerebellum Exterior_GM_Vol', 'Right Cerebellum White Matter_GM_Vol', 'Left Cerebellum White Matter_GM_Vol', 'Right Cerebral White Matter_GM_Vol', 'Left Cerebral White Matter_GM_Vol', 'CSF_GM_Vol', 'Right Hippocampus_GM_Vol', 'Left Hippocampus_GM_Vol', 'Right Inf Lat Vent_GM_Vol', 'Left Inf Lat Vent_GM_Vol', 'Right Lateral Ventricle_GM_Vol', 'Left Lateral Ventricle_GM_Vol', 'Right Pallidum_GM_Vol', 'Left Pallidum_GM_Vol', 'Right Putamen_GM_Vol', 'Left Putamen_GM_Vol', 'Right Thalamus Proper_GM_Vol', 'Left Thalamus Proper_GM_Vol', 'Right Ventral DC_GM_Vol', 'Left Ventral DC_GM_Vol', 'Optic Chiasm_GM_Vol', 'Cerebellar Vermal Lobules I-V_GM_Vol', 'Cerebellar Vermal Lobules VI-VII_GM_Vol', 'Cerebellar Vermal Lobules VIII-X_GM_

In [78]:
from nitk.ml_utils.cross_validation import PredefinedSplit



In [90]:
# cv_test = PredefinedSplit(json_file=config['cv5test'])
CV_DIR="/neurospin/signatures/2025_spetiton_rlink_predict_response_anat/models/study-rlink_mod-cat12vbm_type-roi+age+sex+site_lab-M00_v-4/"

cv_test = PredefinedSplit(json_file=CV_DIR+'supervised_classification_config_cv5test_mskf.json')

# print(cv_stratification_metric(df, factors=factors, cv=cv_test))
print(config['cv5test'])
################################################################################
# Execution function

/neurospin/signatures/2025_spetiton_rlink_predict_response_anat/models/study-rlink_mod-cat12vbm_type-roi+age+sex+site_lab-M00_v-4/supervised_classification_config_cv5test.json


In [79]:
n_splits_val = 5
cv_val = StratifiedKFold(n_splits=n_splits_val, shuffle=True, random_state=42)

In [80]:
residualization_formula = False

if config['residualization']:  
    X, residualizer_estimator, residualization_formula = \
        get_residualizer(data, X, residualization_columns=config['residualization'],
                    print_log=print_log)

print(residualization_formula) #age+sex+site

age+sex+site


In [104]:
# Do not parallelize grid search
models = make_models(n_jobs_grid_search=1, cv_val=cv_val,
                     residualization_formula=residualization_formula,
                     residualizer_estimator=residualizer_estimator)

print("Models (N=%i)" % len(models), models.keys())

metrics_names = ['test_%s' % m for m in config["metrics"]]

Models (N=6) dict_keys(['model-lrl2cv_resid-age+sex+site', 'model-lrenetcv_resid-age+sex+site', 'model-svmrbfcv_resid-age+sex+site', 'model-forestcv_resid-age+sex+site', 'model-gbcv_resid-age+sex+site', 'mlp_cv_resid-age+sex+site'])


In [82]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold



In [65]:
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=4)




In [50]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)

In [83]:
from nitk.python_utils import dict_cartesian_product
from sklearn.model_selection import cross_validate




In [91]:
def cross_validate_wrapper(estimator, cv_test, **kwargs):
    return cross_validate(estimator, X, y, cv=cv_test,
                            scoring=config["metrics"],
                            return_train_score=True, n_jobs=1)

In [85]:
def run_sequential(func, iterable_dict, memory=None,  verbose=0,
                   *args, **kwargs):

    start_time = time.time()
    res = {k:func(*v, verbose=verbose) for k, v in iterable_dict.items()}

    if verbose > 0:
        print('Sequential execution, Elapsed time: \t%.3f sec' %
              (time.time() - start_time))
    
    return res


def run_parallel(func, iterable_dict, memory=None,  verbose=0,
                   *args, **kwargs):

    parallel = Parallel(n_jobs=cpu_count(only_physical_cores=True))

    start_time = time.time()
    res = parallel(delayed(func)(*v, verbose=verbose)
                   for k, v in iterable_dict.items())

    if verbose > 0:
        print('Parallel execution, Elapsed time: \t%.3f sec' %
              (time.time() - start_time))

    return {k:r for k, r in zip(iterable_dict.keys(), res)}

################################################################################
# Repeated CV using cross_validate
# --------------------------------
#
# See `cross_validate <https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#>`_`
# Choose `scoring functions <https://scikit-learn.org/stable/modules/model_evaluation.html#string-name-scorers>`_ 

In [86]:
print(residualizer_estimator)

In [ ]:
rcvs = dict()
rcvs["skf-%i" % 4] = skf
rcvs

models_rcvs = dict_cartesian_product(models, rcvs)
print(models_rcvs)



{('model-lrl2cv_resid-age+sex+site', 'mskf-4'): (Pipeline(steps=[('residualizerestimator',
                 <pylearn_mulm.mulm.residualizer.residualizer.ResidualizerEstimator object at 0x7fb8e5ab91f0>),
                ('standardscaler', StandardScaler()),
                ('gridsearchcv',
                 GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                              estimator=LogisticRegression(class_weight='balanced'),
                              n_jobs=1,
                              param_grid={'C': array([0.001, 0.01 , 0.1  , 1.   ])}))]), PredefinedSplit(json_file=None,
        predefined_splits=[(array([  0,   1, ..., 114, 116]), array([  6,  18,  20,  31,  34,  53,  55,  58,  62,  64,  65,  66,  74,
        85,  88,  91,  96,  98, 106, 109, 110, 113, 115])), (array([  0,   1, ..., 114, 115]), array([  2,   3,   7,   9,  12,  17,  19,  25,  26,  37,  44,  45,  49,
       ...  29,  30,  35,  57,  69,  79,  80,
        81,  84,  89,  9

In [57]:
res = run_parallel(cross_validate_wrapper, models_rcvs, verbose=1)




/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 883, in fit
    return self._fit(
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 649, in _fit
    self._validat

Parallel execution, Elapsed time: 	71.069 sec


In [58]:
res = pd.DataFrame([list(k) + [score[m].mean() for m in metrics_names]
                        for k, score in res.items()],
                    columns=["model", "rep"] + metrics_names)
res = res.sort_values('test_roc_auc', ascending=False)

In [59]:
res

,model,rep,test_accuracy,test_balanced_accuracy,test_roc_auc
0,model-lrl2cv_resid-age+sex+site,skf-4,0.624275,0.614286,0.637989
3,model-forestcv_resid-age+sex+site,skf-4,0.623913,0.519246,0.611799
2,model-svmrbfcv_resid-age+sex+site,skf-4,0.642029,0.545714,0.574127
5,mlp_cv_resid-age+sex+site,skf-4,0.615217,0.485714,0.494365
4,model-gbcv_resid-age+sex+site,skf-4,0.539493,0.489960,0.475079
1,model-lrenetcv_resid-age+sex+site,skf-4,NaN,NaN,NaN


In [105]:
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=4)
df = pd.DataFrame(dict(y=y, site=data['site']))
factors = ['y', 'site']
print(df)
splits_index_mskf = [(train_index, test_index) for train_index, test_index in
                         mskf.split(X, df[factors])]
mskf = PredefinedSplit(splits_index_mskf)
rcvs = dict()
rcvs["mskf-%i" % 4] = mskf
rcvs
models_rcvs = dict_cartesian_product(models, rcvs) #rcvs is a dict with keys 'mskf-'+str(seed) and values are tr/te arrays for 5-fold CV
print(len(models_rcvs))
models_rcvs

     y     site
0    0  site-10
1    1  site-01
2    0  site-07
3    0  site-11
4    0  site-11
..  ..      ...
112  0  site-11
113  1  site-11
114  0  site-07
115  0  site-15
116  0  site-09

[117 rows x 2 columns]
6


{('model-lrl2cv_resid-age+sex+site',
  'mskf-4'): (Pipeline(steps=[('residualizerestimator',
                   <pylearn_mulm.mulm.residualizer.residualizer.ResidualizerEstimator object at 0x7fb8e5bddbe0>),
                  ('standardscaler', StandardScaler()),
                  ('gridsearchcv',
                   GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                                estimator=LogisticRegression(class_weight='balanced'),
                                n_jobs=1,
                                param_grid={'C': array([0.001, 0.01 , 0.1  , 1.   ])}))]), PredefinedSplit(json_file=None,
          predefined_splits=[(array([  0,   1, ..., 114, 116]), array([  6,  18,  20,  31,  34,  53,  55,  58,  62,  64,  65,  66,  74,
          85,  88,  91,  96,  98, 106, 109, 110, 113, 115])), (array([  0,   1, ..., 114, 115]), array([  2,   3,   7,   9,  12,  17,  19,  25,  26,  37,  44,  45,  49,
         ...  29,  30,  35,  57,  69,  79,  80,
  

In [106]:
res = run_parallel(cross_validate_wrapper, models_rcvs, verbose=1)


/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 883, in fit
    return self._fit(
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 649, in _fit
    self._validat

Parallel execution, Elapsed time: 	26.858 sec


In [107]:
res = pd.DataFrame([list(k) + [score[m].mean() for m in metrics_names]
                    for k, score in res.items()],
                columns=["model", "rep"] + metrics_names)
res = res.sort_values('test_roc_auc', ascending=False)
res

,model,rep,test_accuracy,test_balanced_accuracy,test_roc_auc
3,model-forestcv_resid-age+sex+site,mskf-4,0.640217,0.530238,0.715563
0,model-lrl2cv_resid-age+sex+site,mskf-4,0.640217,0.613849,0.680473
4,model-gbcv_resid-age+sex+site,mskf-4,0.608333,0.585893,0.647247
2,model-svmrbfcv_resid-age+sex+site,mskf-4,0.649638,0.593194,0.625913
5,mlp_cv_resid-age+sex+site,mskf-4,0.598188,0.478155,0.489208
1,model-lrenetcv_resid-age+sex+site,mskf-4,NaN,NaN,NaN


In [34]:
def cv_stratification_metric(df, factors, cv):
    """Metrics (SSE) that evaluate the quality kfolds stratification for many
    factors. Sum accross fold SSE (true proportions (from df[factors], 
    fold proportions (from df[test, factors])

    Parameters
    ----------
    df : _type_
        _description_
    factors : _type_
        _description_
    cv : _type_
        _description_
    """
    def proportions_byfactors(df, factors):
        counts = df.groupby(factors).size()
        prop = counts / counts.sum()
        return prop

    prop_tot = proportions_byfactors(df=df, factors=factors)

    sse = 0
    for train_index, test_index in cv.split(X, y):
        prop_fold = proportions_byfactors(df=df.iloc[test_index], factors=factors)
        #print(np.sum((prop_tot - prop_fold) ** 2))
        sse += np.sum((prop_tot - prop_fold) ** 2)

    return sse

In [35]:
rcvs = dict()
sse = []

for seed in range(10):
    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    splits_index_mskf = [(train_index, test_index) for train_index, test_index in
                         mskf.split(X, df[factors])]
    # warning with MultilabelStratifiedKFold: here it works because site and y are catagorical but it would require binarization
    # if another (continuous) variable was added to the list of factors (like age)
    mskf = PredefinedSplit(splits_index_mskf)
    rcvs["mskf-%i" % seed] = mskf

    sse_ = cv_stratification_metric(df, factors=factors, cv=mskf)
    sse.append(["mskf", seed, sse_])

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    #splits_index_skf = [test_index for train_index, test_index in skf.split(X, y)]
    rcvs["skf-%i" % seed] = skf
    sse_ = cv_stratification_metric(df, factors=factors, cv=skf)
    sse.append(["skf", seed, sse_])
    

In [ ]:
models_rcvs = dict_cartesian_product(models, rcvs) # rcvs is a dict with keys 'mskf-'+str(seed) and values are tr/te arrays for 5-fold CV
print(len(models_rcvs))
res = run_parallel(cross_validate_wrapper, models_rcvs, verbose=1)


120


/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 883, in fit
    return self._fit(
  File "/home/sp272479/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 649, in _fit
    self._validat

KeyboardInterrupt: 

In [ ]:
res = pd.DataFrame([list(k) + [score[m].mean() for m in metrics_names]
                    for k, score in res.items()],
                columns=["model", "rep"] + metrics_names)
res = res.sort_values('test_roc_auc', ascending=False)
res